In [ ]:
import pathlib
from typing import List, Dict, Any, Tuple

In [ ]:
RANK = 4


class ExpDir:
    def __init__(self, exp_dir: pathlib.Path):
        self.exp_dir = exp_dir

        # parse exp_dir name: <dataset>-<config>-<seq_len>
        self.name = str(exp_dir.name)

        # replace '--1' with '-0', special case for no window
        self.name = self.name.replace("--1", "-0")

        self.dataset, self.config, self.seq_len, self.model, self.window = (
            self.name.split("-")
        )
        self.seq_len = int(self.seq_len)
        self.res = self.read_res()
        self.key = self._get_key()

    def __repr__(self):
        return f"ExpDir({self.exp_dir.name}), seq_len={self.seq_len}"

    @staticmethod
    def format_res(res: str) -> float:
        # bytes to GB
        # return round(float(res) / 1024 / 1024 / 1024, 3)
        return float(res)

    def read_single_res(self, res_file: pathlib.Path) -> float:
        with open(res_file, "r") as f:
            res = f.read()
        return self.format_res(res)

    def read_res(self) -> List[int]:
        res_list = []
        for rank in range(RANK):
            res_file = self.exp_dir / f"mem-{rank}.txt"
            # assert res_file.exists(), f'{res_file} not exists'
            if not res_file.exists():
                print(f"[warning]: {res_file} not exists, return empty list")
                return []
            res_list.append(self.read_single_res(res_file))
        return res_list

    def _get_key(self) -> Tuple:
        return self.dataset, self.config, self.seq_len, self.model

In [ ]:
INTERESTED_RANK = 0

base_dir = pathlib.Path("../output")
group_dir_names = ["20231112_5", "20231112_4"]
group_dir_exp_list = []
for group_dir_name in group_dir_names:
    for edir in (base_dir / group_dir_name).iterdir():
        if edir.is_dir():
            group_dir_exp_list.append(ExpDir(edir))
counter = {}  # key -> set()
for exp in group_dir_exp_list:
    if exp.key not in counter:
        counter[exp.key] = set()
    counter[exp.key].add(exp.res[INTERESTED_RANK])

for key, res_set in counter.items():
    print(key, res_set)